In [1]:
import threading
import os

In [2]:
def task1():
    print("Task 1 assigned to thread: {}".format(threading.current_thread().name))
    print("ID of process running task 1: {}".format(os.getpid()))

def task2():
    print("Task 2 assigned to thread: {}".format(threading.current_thread().name))
    print("ID of process running task 2: {}".format(os.getpid()))

if __name__ == "__main__":

    print("ID of process running main program: {}".format(os.getpid()))

    print("Main thread name: {}".format(threading.current_thread().name))

    t1 = threading.Thread(target=task1, name='t1')
    t2 = threading.Thread(target=task2, name='t2')

    t1.start()
    t2.start()

    t1.join()
    t2.join()

ID of process running main program: 22324
Main thread name: MainThread
Task 1 assigned to thread: t1
ID of process running task 1: 22324
Task 2 assigned to thread: t2
ID of process running task 2: 22324


In [3]:
import threading
import time

# Create a pause event
pause_event = threading.Event()
pause_event.set()  # Start as "not paused"

def worker():
    count = 0
    while count < 10:
        pause_event.wait()  # Will pause here if event is cleared
        print(f"Working... {count}")
        time.sleep(1)
        count += 1
    print("Thread finished.")

# Create and start the thread
t = threading.Thread(target=worker)
t.start()

# Simulate pause/resume
time.sleep(3)
print("Pausing thread...")
pause_event.clear()  # Pause the thread

time.sleep(5)
print("Resuming thread...")
pause_event.set()  # Resume the thread

t.join()
print("Main program done.")


Working... 0
Working... 1
Working... 2
Pausing thread...
Resuming thread...
Working... 3
Working... 4
Working... 5
Working... 6
Working... 7
Working... 8
Working... 9
Thread finished.
Main program done.


In [4]:
import tkinter as tk
from threading import Thread, Event
import time

class MusicPlayerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🎵 Music Player")
        self.root.geometry("300x200")

        self.status_label = tk.Label(root, text="Status: Stopped", font=('Arial', 12))
        self.status_label.pack(pady=10)

        self.play_button = tk.Button(root, text="▶️ Play", command=self.play)
        self.play_button.pack(fill='x', padx=20, pady=5)

        self.pause_button = tk.Button(root, text="⏸️ Pause", command=self.pause)
        self.pause_button.pack(fill='x', padx=20, pady=5)

        self.stop_button = tk.Button(root, text="⏹️ Stop", command=self.stop)
        self.stop_button.pack(fill='x', padx=20, pady=5)

        self.progress_label = tk.Label(root, text="Progress: 0s / 10s")
        self.progress_label.pack(pady=10)

        self.pause_event = Event()
        self.stop_event = Event()
        self.thread = None

    def play(self):
        if self.thread is None or not self.thread.is_alive():
            self.pause_event.set()
            self.stop_event.clear()
            self.status_label.config(text="Status: Playing")
            self.thread = Thread(target=self._play_music)
            self.thread.start()
        else:
            self.status_label.config(text="Status: Playing")
            self.pause_event.set()  # Resume

    def pause(self):
        self.status_label.config(text="Status: Paused")
        self.pause_event.clear()

    def stop(self):
        self.status_label.config(text="Status: Stopped")
        self.stop_event.set()
        self.pause_event.set()

    def _play_music(self):
        total_duration = 10
        for second in range(total_duration):
            if self.stop_event.is_set():
                break
            self.pause_event.wait()
            time.sleep(1)
            self.progress_label.config(text=f"Progress: {second + 1}s / {total_duration}s")
        else:
            self.status_label.config(text="Status: Finished")

# -------- Run the App --------
root = tk.Tk()
app = MusicPlayerApp(root)
root.mainloop()
